# O conjunto de dados utilizados para criação desse modelo foi o Chars74k

Download:
- https://info-ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishFnt.tgz
- https://info-ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz

O script abaixo utiliza as bibliotecas padrão do Python (os, shutil) para manipulação de arquivos e a scikit-learn (sklearn) para realizar a crucial separação estratificada dos dados (80% Treinamento, 10% Validação, 10% Teste).

Pré-requisitos: ter as bibliotecas numpy e scikit-learn instaladas:
- pip install numpy scikit-learn

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. CONFIGURAÇÃO DE DIRETÓRIOS ---
# Diretório onde as pastas EnglishFnt e EnglisgImg estão localizadas
BASE_DIR = os.getcwd() 

# Diretório final de saída
OUTPUT_DIR = os.path.join(BASE_DIR, 'chars74k_dataset_final') 

# Diretórios de origem do dataset Chars74K
FNT_DIR = os.path.join(BASE_DIR, 'EnglishFnt', 'English', 'Fnt')
IMG_DIR = os.path.join(BASE_DIR, 'EnglishImg', 'English', 'Img') 

# Proporções de divisão (Treino/Validação/Teste)
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

# --- 2. MAPEAR NOMES DE PASTAS 'SampleXXX' PARA RÓTULOS REAIS ---
# O Chars74K usa Sample001-010 para dígitos (0-9), Sample011-036 para maiúsculas (A-Z)
# e Sample037-062 para minúsculas (a-z).
def get_label_map():
    label_map = {}
    
    # Dígitos (0-9)
    for i in range(1, 11):
        label_map[f'Sample{i:03d}'] = str(i - 1)

    # Letras Maiúsculas (A-Z)
    for i in range(11, 37):
        char = chr(ord('A') + i - 11)
        label_map[f'Sample{i:03d}'] = char

    # Letras Minúsculas (a-z)
    for i in range(37, 63):
        char = chr(ord('a') + i - 37)
        label_map[f'Sample{i:03d}'] = char
        
    return label_map

# --- 3. FUNÇÃO PRINCIPAL DE COLETA DE IMAGENS ---
def collect_all_images(base_path, label_map):
    all_files = []
    all_labels = []
    
    print(f"Coletando imagens de: {base_path}")
    
    # Iterar sobre todas as subpastas SampleXXX
    for sample_dir_name in os.listdir(base_path):
        if not sample_dir_name.startswith('Sample'):
            continue
            
        sample_path = os.path.join(base_path, sample_dir_name)
        if not os.path.isdir(sample_path):
            continue
            
        # Obter o rótulo real (ex: '0', 'A', 'a')
        label = label_map.get(sample_dir_name)
        if not label:
            print(f"Aviso: Sample ID {sample_dir_name} não encontrado no mapeamento.")
            continue
            
        # Coletar todos os arquivos .png ou .bmp dentro do diretório SampleXXX
        for filename in os.listdir(sample_path):
            if filename.endswith(('.png', '.bmp')):
                file_path = os.path.join(sample_path, filename)
                all_files.append(file_path)
                all_labels.append(label)
                
    return all_files, all_labels

# --- 4. FUNÇÃO PARA CRIAR E COPIAR OS ARQUIVOS ---
def create_and_copy(file_list, label_list, set_name):
    print(f"\nCopiando {len(file_list)} imagens para o conjunto de {set_name}...")
    
    # Criar a pasta raiz do conjunto (train, validation, ou test)
    set_dir = os.path.join(OUTPUT_DIR, set_name)
    os.makedirs(set_dir, exist_ok=True)
    
    for file_path, label in zip(file_list, label_list):
        # 1. Criar a subpasta da classe dentro do conjunto (ex: train/A)
        target_class_dir = os.path.join(set_dir, label)
        os.makedirs(target_class_dir, exist_ok=True)
        
        # 2. Copiar o arquivo
        shutil.copy(file_path, target_class_dir)

# --- EXECUÇÃO PRINCIPAL ---
if __name__ == '__main__':
    
    # Limpeza e criação do diretório de saída
    if os.path.exists(OUTPUT_DIR):
        print(f"Removendo diretório existente: {OUTPUT_DIR}")
        shutil.rmtree(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR)
    
    label_map = get_label_map()
    
    # A) Coletar dados sintéticos (EnglishFnt)
    fnt_files, fnt_labels = collect_all_images(FNT_DIR, label_map)
    
    # B) Coletar dados naturais (EnglisgImg)
    # Ignorando Msk (máscaras) e focando em Bmp (imagens) dos diretórios GoodImg e BadImag
    img_files, img_labels = [], []
    for quality_dir in ['GoodImg', 'BadImag']:
        for type_dir in ['Bmp']: # Focar apenas em Bmp (imagens), ignorar Msk (máscaras)
            path = os.path.join(IMG_DIR, quality_dir, type_dir)
            files, labels = collect_all_images(path, label_map)
            img_files.extend(files)
            img_labels.extend(labels)
    
    # C) Consolidar TODOS os arquivos em uma única lista
    all_files = fnt_files + img_files
    all_labels = fnt_labels + img_labels
    
    if not all_files:
        print("ERRO: Nenhuma imagem encontrada. Verifique os caminhos de diretório.")
    else:
        print(f"\nTOTAL de imagens coletadas: {len(all_files)}")
        
        # 5. DIVISÃO ESTRATIFICADA (Duas Etapas)
        
        # Passo 1: Treinamento vs. Teste/Validação (80% vs 20%)
        X_train, X_temp, y_train, y_temp = train_test_split(
            all_files, all_labels, 
            test_size=(VAL_RATIO + TEST_RATIO), 
            random_state=42, 
            shuffle=True, 
            stratify=all_labels
        )

        # Passo 2: Validação vs. Teste (10% vs 10%)
        # Calcula o novo ratio de teste em relação ao conjunto temporário (0.5 = 10% / 20%)
        test_size_ratio = TEST_RATIO / (VAL_RATIO + TEST_RATIO) 
        
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, 
            test_size=test_size_ratio, 
            random_state=42, 
            shuffle=True, 
            stratify=y_temp
        )
        
        print("\n--- RESUMO DA DIVISÃO ESTRATIFICADA ---")
        print(f"Treinamento: {len(X_train)} imagens ({len(X_train)/len(all_files):.1%})")
        print(f"Validação: {len(X_val)} imagens ({len(X_val)/len(all_files):.1%})")
        print(f"Teste: {len(X_test)} imagens ({len(X_test)/len(all_files):.1%})")
        print("---------------------------------------")
        
        # 6. COPIAR ARQUIVOS PARA A ESTRUTURA FINAL
        create_and_copy(X_train, y_train, 'train')
        create_and_copy(X_val, y_val, 'validation')
        create_and_copy(X_test, y_test, 'test')
        
        print("\n✅ Organização concluída! O dataset final está em 'chars74k_dataset_final'.")

Removendo diretório existente: /home/carlos/classificacao_segmentacao/chars74k_dataset_final
Coletando imagens de: /home/carlos/classificacao_segmentacao/EnglishFnt/English/Fnt
Coletando imagens de: /home/carlos/classificacao_segmentacao/EnglishImg/English/Img/GoodImg/Bmp
Coletando imagens de: /home/carlos/classificacao_segmentacao/EnglishImg/English/Img/BadImag/Bmp

TOTAL de imagens coletadas: 75495

--- RESUMO DA DIVISÃO ESTRATIFICADA ---
Treinamento: 60396 imagens (80.0%)
Validação: 7549 imagens (10.0%)
Teste: 7550 imagens (10.0%)
---------------------------------------

Copiando 60396 imagens para o conjunto de train...

Copiando 7549 imagens para o conjunto de validation...

Copiando 7550 imagens para o conjunto de test...

✅ Organização concluída! O dataset final está em 'chars74k_dataset_final'.
